In [3]:
!pip install torch transformers pandas numpy faiss-cpu tqdm


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import pandas as pd

# 1. Recriando o dataframe base com seus dados
dados_reais = [
    ("Não reconheço essa compra na padaria", "Cartão | Contestação | Compra Não Reconhecida"),
    ("Apareceu uma assinatura de streaming que eu cancelei", "Cartão | Contestação | Compra Recorrente"),
    ("Clonaram meu cartão, tem gastos em dolar", "Cartão | Contestação | Fraude Externa"),
    ("Minha fatura veio com anuidade sendo que era isento", "Cartão | Tarifas | Cobrança de Anuidade"),
    ("Quero aumentar meu limite para viajar", "Cartão | Limite | Análise de Crédito"),
    ("O cartão chegou mas a senha não funciona", "Cartão | Entrega | Desbloqueio/Senha"),
    ("Recebi um cartão que eu nunca pedi", "Cartão | Contratação | Cartão não Solicitado"),
    ("Meu pix saiu da conta mas a loja diz que não recebeu", "Conta Corrente | Pix | Falha Tecnológica"),
    ("Fiz um pix errado, quero o dinheiro de volta", "Conta Corrente | Pix | Devolução (MED)"),
    ("Tem uma tarifa de cesta de serviços que não contratei", "Conta Corrente | Tarifas | Pacote de Serviços"),
    ("Não consigo acessar minha conta pelo celular", "Canais Digitais | App | Erro de Acesso"),
    ("O app fecha sozinho quando tento pagar boleto", "Canais Digitais | App | Instabilidade"),
    ("Quero encerrar minha conta, vou mudar de banco", "Conta Corrente | Encerramento | Cancelamento Voluntário"),
    ("Vocês bloquearam minha conta sem avisar", "Conta Corrente | Bloqueio | Segurança/Compliance"),
    ("Os juros desse empréstimo estão abusivos", "Empréstimo | Pessoal | Revisão de Juros"),
    ("Quero quitar meu financiamento de carro antecipado", "Empréstimo | Veículos | Amortização"),
    ("Estou recebendo ligações de cobrança o dia todo", "Cobrança | Abordagem | Excesso de Ligações"),
    ("Quero renegociar minha dívida, estou desempregado", "Cobrança | Renegociação | Acordo"),
    ("Descontaram a parcela do emprestimo duas vezes", "Empréstimo | Consignado | Desconto Indevido"),
    ("O gerente foi extremamente grosso comigo", "Atendimento | Agência | Postura do Funcionário"),
    ("Fiquei 2 horas na fila esperando atendimento", "Atendimento | Agência | Tempo de Espera"),
    ("O SAC desliga a chamada na minha cara", "Atendimento | SAC | Qualidade da Ligação"),
    ("Ninguém resolve meu problema, ficam jogando de um para o outro", "Atendimento | Geral | Falta de Resolução"),
    ("Quero resgatar meu dinheiro da poupança e não consigo", "Investimentos | Poupança | Resgate"),
    ("Meu assessor de investimentos me recomendou algo que deu prejuízo", "Investimentos | Assessoria | Suitability"),
    ("Não recebi o informe de rendimentos para o imposto de renda", "Investimentos | Documentação | Informe de Rendimentos")
]

df_historico = pd.DataFrame(dados_reais, columns=['TEXTO_CLIENTE', 'TARGET_FULL'])

# ==========================================================
# GERAÇÃO AUTOMÁTICA DO DF_CATEG
# ==========================================================

# 1. Pega apenas os valores únicos da coluna TARGET_FULL (remove duplicatas)
#    e ordena alfabeticamente para ficar organizado.
unique_targets = df_historico['TARGET_FULL'].drop_duplicates().sort_values()

# 2. Divide a string usando o separador " | "
#    expand=True transforma a lista quebrada em novas colunas
df_split = unique_targets.str.split(' \| ', expand=True)

# 3. Renomeia as colunas e cria o df_categ final
df_categ = df_split.rename(columns={0: 'FAMILIA', 1: 'PRODUTO', 2: 'ASSUNTO'})

# 4. (Opcional) Reseta o índice para ficar limpo
df_categ = df_categ.reset_index(drop=True)

print(f"Total de categorias únicas encontradas: {len(df_categ)}")
print("\n--- Visualização do df_categ ---")

# Formatação para exibir todas as colunas alinhadas
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 1000)
df_categ

Total de categorias únicas encontradas: 26

--- Visualização do df_categ ---


,FAMILIA,PRODUTO,ASSUNTO
0,Atendimento,Agência,Postura do Funcionário
1,Atendimento,Agência,Tempo de Espera
2,Atendimento,Geral,Falta de Resolução
3,Atendimento,SAC,Qualidade da Ligação
4,Canais Digitais,App,Erro de Acesso
5,Canais Digitais,App,Instabilidade
6,Cartão,Contestação,Compra Não Reconhecida
7,Cartão,Contestação,Compra Recorrente
8,Cartão,Contestação,Fraude Externa
9,Cartão,Contratação,Cartão não Solicitado


In [7]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm  # Barra de progresso

# Tenta importar FAISS, senão usa fallback
try:
    import faiss
    USE_FAISS = True
except ImportError:
    USE_FAISS = False
    print("Aviso: FAISS não encontrado. Usando multiplicação de matrizes (mais lento, mas funciona).")

# ==========================================
# 1. CONFIGURAÇÕES E CARREGAMENTO
# ==========================================
MODEL_PATH = "./bert_final"  # Pasta onde você reconstruiu o modelo
BATCH_SIZE = 32              # Processar 32 textos por vez (ajuste conforme memória RAM)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Carregando modelo de: {MODEL_PATH}")
print(f"Dispositivo de processamento: {DEVICE.upper()}")

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModel.from_pretrained(MODEL_PATH).to(DEVICE)
model.eval() # Modo de avaliação (mais rápido, desliga dropout)

# ==========================================
# 2. FUNÇÕES OTIMIZADAS
# ==========================================

def generate_embeddings_batched(texts, tokenizer, model, batch_size=32):
    """Gera embeddings em lotes para não estourar a memória e ser rápido."""
    all_embeddings = []
    
    # Loop com barra de progresso
    for i in tqdm(range(0, len(texts), batch_size), desc="Gerando Embeddings"):
        batch_texts = texts[i:i + batch_size]
        
        # Tokenização
        inputs = tokenizer(
            batch_texts, 
            padding=True, 
            truncation=True, 
            max_length=128, 
            return_tensors="pt"
        ).to(DEVICE)
        
        with torch.no_grad():
            outputs = model(**inputs)
            
        # Mean Pooling (Média da última camada)
        # Atenção: Mover para CPU para liberar VRAM/RAM
        embeddings = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
        
        # Normalização L2 (Essencial para Cosseno funcionar com FAISS/MatMul)
        # Isso faz com que a Distância Euclidiana seja equivalente à Similaridade de Cosseno
        faiss.normalize_L2(embeddings) if USE_FAISS else None
        if not USE_FAISS:
            norm = np.linalg.norm(embeddings, axis=1, keepdims=True)
            embeddings = embeddings / (norm + 1e-10)
            
        all_embeddings.append(embeddings)
        
    return np.vstack(all_embeddings)

# ==========================================
# 3. PREPARAÇÃO DOS DADOS
# ==========================================

# --- A) Carregar seus Dataframes reais aqui ---
# df_categ = pd.read_csv("sua_base_categorias.csv")
# df_manif = pd.read_csv("sua_base_reclamacoes.csv")

# DF_CATEG CRIADO ACIMA

df_manif = pd.DataFrame({
    'ID': range(1001, 1017), # IDs fictícios de 1001 a 1015
    'TEXTO_CLIENTE': [
        # --- Variações de CARTÃO (Treino: "Não reconheço compra") ---
        "Apareceu uma cobrança da Netflix na minha fatura que eu não assinei.",
        "Tem um débito de 500 reais do Mercado Livre que é fraude.",
        "Veio uma compra internacional que desconheço.",

        # --- Variações de PIX (Treino: "Pix saiu mas não chegou") ---
        "Fiz uma transferência instantânea pro meu irmão e o dinheiro sumiu.",
        "O comprovante do pix saiu, mas o destinatário diz que não recebeu nada.",
        "Erro na transação, debitou o saldo mas não completou o envio.",

        # --- Variações de CANCELAMENTO (Treino: "Quero cancelar conta") ---
        "Gostaria de encerrar meu relacionamento com este banco.",
        "Desejo fechar minha conta, não uso mais.",
        "Qual o procedimento para o encerramento da conta corrente?",

        # --- Variações de ATENDIMENTO (Treino: "Gerente mal educado") ---
        "O atendente do caixa foi extremamente grosso comigo hoje.",
        "Fui destratado na agência da Avenida Paulista.",
        "Falta de respeito por parte dos funcionários da agência.",

        # --- Variações de JUROS (Treino: "Juros abusivos") ---
        "A taxa que vocês estão cobrando no cheque especial é um absurdo.",
        "Descontaram uns encargos da minha conta que eu não concordo.",
        "O valor dos juros rotativos está muito alto este mês.",
        
        # --- CASO AMBÍGUO (Teste de Stress) ---
        "Vocês estão roubando meu dinheiro com essas taxas no cartão." 
        # (Mistura "Cartão" com "Taxas/Juros" - vamos ver o que ele decide)
    ]
})

# --- B) Criar "Assinatura Hierárquica" das Categorias ---
print("Preparando hierarquia das categorias...")
df_categ['texto_full'] = df_categ['FAMILIA'] + " " + df_categ['PRODUTO'] + " " + df_categ['ASSUNTO']

# ==========================================
# 4. GERAÇÃO DOS VETORES (A parte pesada)
# ==========================================

print(f"\n1. Vetorizando {len(df_categ)} Categorias...")
categ_vectors = generate_embeddings_batched(df_categ['texto_full'].tolist(), tokenizer, model, BATCH_SIZE)

print(f"\n2. Vetorizando {len(df_manif)} Reclamações...")
manif_vectors = generate_embeddings_batched(df_manif['TEXTO_CLIENTE'].tolist(), tokenizer, model, BATCH_SIZE)

# ==========================================
# 5. BUSCA DE SIMILARIDADE (Otimização FAISS)
# ==========================================
print("\n3. Realizando o Matching (Cruzamento)...")

# Índices das melhores categorias
best_indices = []
confidence_scores = []

if USE_FAISS:
    # Cria um índice Flat (busca exata) com produto interno (IP)
    # Como os vetores estão normalizados, IP == Cosseno
    dimension = categ_vectors.shape[1]
    index = faiss.IndexFlatIP(dimension)
    
    # Adiciona as categorias ao "banco de dados" do FAISS
    index.add(categ_vectors)
    
    # Busca os 1 vizinhos mais próximos para cada reclamação
    # k=1 retorna a melhor categoria
    D, I = index.search(manif_vectors, k=1)
    
    best_indices = I.flatten()
    confidence_scores = D.flatten()

else:
    # Fallback: Multiplicação de Matrizes (Rápido, mas consome muita RAM)
    # Similaridade = A . B^T
    print("Usando Numpy Dot Product...")
    sim_matrix = np.dot(manif_vectors, categ_vectors.T)
    best_indices = np.argmax(sim_matrix, axis=1)
    confidence_scores = np.max(sim_matrix, axis=1)

# ==========================================
# 6. ATRIBUIÇÃO DOS RESULTADOS
# ==========================================

print("Atribuindo resultados ao dataframe...")

# Usamos os índices encontrados para buscar as linhas correspondentes no df_categ
matched_categories = df_categ.iloc[best_indices].reset_index(drop=True)

# Acopla ao dataframe original
df_final = pd.concat([df_manif, matched_categories[['FAMILIA', 'PRODUTO', 'ASSUNTO']]], axis=1)
df_final['SCORE_CONFIANCA'] = confidence_scores

# Opcional: Filtrar apenas classificações com confiança alta
# df_final = df_final[df_final['SCORE_CONFIANCA'] > 0.85]

print("\n--- AMOSTRA DO RESULTADO ---")
df_final[['TEXTO_CLIENTE', 'FAMILIA', 'PRODUTO', 'ASSUNTO', 'SCORE_CONFIANCA']]


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\felip\AppData\Local\Programs\Python\Python311\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\felip\AppData\Local\Programs\Python\Python311\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\felip\AppData\Local\Programs\Python\Python311\Lib\site-packages\ipykernel\kernelapp.py", 

AttributeError: _ARRAY_API not found

Aviso: FAISS não encontrado. Usando multiplicação de matrizes (mais lento, mas funciona).
Carregando modelo de: ./bert_final
Dispositivo de processamento: CPU
Preparando hierarquia das categorias...

1. Vetorizando 26 Categorias...


Gerando Embeddings: 100%|██████████| 1/1 [00:00<00:00,  2.34it/s]



2. Vetorizando 16 Reclamações...


Gerando Embeddings: 100%|██████████| 1/1 [00:00<00:00,  3.72it/s]


3. Realizando o Matching (Cruzamento)...
Usando Numpy Dot Product...
Atribuindo resultados ao dataframe...

--- AMOSTRA DO RESULTADO ---


,TEXTO_CLIENTE,FAMILIA,PRODUTO,ASSUNTO,SCORE_CONFIANCA
0,Apareceu uma cobrança da Netflix na minha fatura que eu não assinei.,Cartão,Contestação,Compra Não Reconhecida,0.617362
1,Tem um débito de 500 reais do Mercado Livre que é fraude.,Cartão,Contestação,Compra Não Reconhecida,0.600686
2,Veio uma compra internacional que desconheço.,Cartão,Contestação,Compra Não Reconhecida,0.591427
3,Fiz uma transferência instantânea pro meu irmão e o dinheiro sumiu.,Cartão,Entrega,Desbloqueio/Senha,0.554502
4,"O comprovante do pix saiu, mas o destinatário diz que não recebeu nada.",Cartão,Contestação,Compra Não Reconhecida,0.557754
5,"Erro na transação, debitou o saldo mas não completou o envio.",Cartão,Contestação,Compra Não Reconhecida,0.621736
6,Gostaria de encerrar meu relacionamento com este banco.,Atendimento,Geral,Falta de Resolução,0.563890
7,"Desejo fechar minha conta, não uso mais.",Cartão,Contestação,Compra Não Reconhecida,0.519100
8,Qual o procedimento para o encerramento da conta corrente?,Conta Corrente,Pix,Devolução (MED),0.637285
9,O atendente do caixa foi extremamente grosso comigo hoje.,Atendimento,Geral,Falta de Resolução,0.568711
